# Data Preparation


In [1]:
# --------------------------------------------------------------------------------------------------------
# ciaFactBook.ipynb -- data preparation, inspection, and visualization 
# --------------------------------------------------------------------------------------------------------
# 0.01|03.01.22|AB|creation
# 0.02|16.01.22|AB|filter Europe, cleanse data, save as csv
# 0.03|18.01.22|AB|join with other tables, enhancements
# --------------------------------------------------------------------------------------------------------

# initialize

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#from datetime import datetime, date 
import re # regular expression
import seaborn as sns

PATH_DATA = r'../../data/'
INTERACTIVE = False
VALUE_COLNAME = 'language'  # name for new column containing value

if INTERACTIVE:
    %matplotlib notebook
else:
    %matplotlib inline 

## Languages per country (official, minority)

_The number of persons having their usual residence in a country on 1 January of the respective year. When usually resident population is not available, countries may report legal or registered residents._

https://www.cia.gov/the-world-factbook/about/archives/


In [2]:
fileName = PATH_DATA + 'countries_languages_all.csv'
# header = 1st line
data = pd.read_csv(fileName, sep=',', header=0) 

In [3]:
data.head()

,Country,Recognized Languages
0,Afghanistan,Afghan Persian or Dari (official) 77% (Dari fu...
1,Albania,Albanian 98.8% (official - derived from Tosk d...
2,Algeria,"Arabic (official), French (lingua franca), Ber..."
3,Andorra,"Catalan (official), French, Castilian, Portuguese"
4,Angola,"Portuguese 71.2% (official), Umbundu 23%, Kiko..."


Non-European countries to be  filtered out.
Second (unstructured) column is transformed as follows:
- Split text into 1 column per language
- add one column for share / official (yes/no)

In [4]:
# data['Recognized Languages'].str.extract([A-Z][a-z]+ )
# split column into column per language
data = data.join(data['Recognized Languages'].str.split(',', expand=True))

In [5]:
# drop split column
data.pop('Recognized Languages')
data.head()

,Country,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Afghanistan,Afghan Persian or Dari (official) 77% (Dari fu...,Pashto (official) 48%,Uzbek 11%,English 6%,Turkmen 3%,Urdu 3%,Pashayi 1%,Nuristani 1%,Arabic 1%,Balochi 1% (2017 est.),None,None,None,None,None,None,None,None,None
1,Albania,Albanian 98.8% (official - derived from Tosk d...,Greek 0.5%,other 0.6% (including Macedonian,Romani,Vlach,Turkish,Italian,and Serbo-Croatian),unspecified 0.1% (2011 est.),None,None,None,None,None,None,None,None,None,None
2,Algeria,Arabic (official),French (lingua franca),Berber or Tamazight (official); dialects incl...,Shawiya Berber (Tacawit),Mzab Berber,Tuareg Berber (Tamahaq),None,None,None,None,None,None,None,None,None,None,None,None,None
3,Andorra,Catalan (official),French,Castilian,Portuguese,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Angola,Portuguese 71.2% (official),Umbundu 23%,Kikongo 8.2%,Kimbundu 7.8%,Chokwe 6.5%,Nhaneca 3.4%,Nganguela 3.1%,Fiote 2.4%,Kwanhama 2.3%,Muhumbi 2.1%,Luvale 1%,other 3.6% (2014 est.),None,None,None,None,None,None,None


In [6]:
# create a row for each country-language combination (pivot wide -> long)
# key is country (for each language and country, a row is to be created)
#data = 
data = data.melt(['Country'], value_name = 'languageRaw', var_name = 'langId')
#, var_name = 'langId', value_vars = None, value_name = None)   #, value_name = VALUE_COLNAME)
# remove empty languages
data = data.dropna()
# note: the number of languages spoken in a given country is equal to max(langId)+1; 
# most languages are spoken in Zambia (18)
data.head()

,Country,langId,languageRaw
0,Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...
1,Albania,0,Albanian 98.8% (official - derived from Tosk d...
2,Algeria,0,Arabic (official)
3,Andorra,0,Catalan (official)
4,Angola,0,Portuguese 71.2% (official)


In [7]:
# European languages only
# join with european_languages.csv
eur_lang = pd.read_csv("../../data/general/european_languages.csv")
eur_lang.head()

,LangID,CountryID,LangStatus,Name_lang,Name_country,Area
0,aae,IT,L,"Albanian, Arbëreshë",Italy,Europe
1,cim,IT,L,Cimbrian,Italy,Europe
2,egl,IT,L,Emilian,Italy,Europe
3,fur,IT,L,Friulian,Italy,Europe
4,ils,IT,L,International Sign,Italy,Europe


In [8]:
# semi join with European languages
data = data[data.Country.isin(eur_lang.Name_country)]
data.head()

,Country,langId,languageRaw
1,Albania,0,Albanian 98.8% (official - derived from Tosk d...
9,Austria,0,German (official nationwide) 88.6%
15,Belarus,0,Russian (official) 70.2%
16,Belgium,0,Dutch (official) 60%
21,Bosnia and Herzegovina,0,Bosnian (official) 52.9%


In [9]:
# -- create an index
data.set_index(['Country', 'langId'], inplace = True)

In [10]:
# mark official languages (new column official [Boolean])
pattern = "official"  
# data[data['languageRaw'].str.contains(pattern)]
data['official'] = data['languageRaw'].str.contains(pattern)
data

,,languageRaw,official
Country,langId,,
Albania,0,Albanian 98.8% (official - derived from Tosk d...,True
Austria,0,German (official nationwide) 88.6%,True
Belarus,0,Russian (official) 70.2%,True
Belgium,0,Dutch (official) 60%,True
Bosnia and Herzegovina,0,Bosnian (official) 52.9%,True
...,...,...,...
Luxembourg,8,other 8.4% (2011 est.),False
Spain,8,000 speakers),False
Switzerland,8,Romansh (official) 0.5%,True


In [11]:
# Extract actual language name, identified as the first capitalized word (this heuristic introduces some NA's)
pattern = r'(?P<language>[A-Z][a-z]+)'  # named group becomes new column 'language'
#data['languageRaw'].str.extract(pattern)
data = data.join(data['languageRaw'].str.extract(pattern))
data = data.dropna()
data.sort_values(by = ["Country", "langId"], inplace=True)

In [12]:
# data cleansing
data.drop(("Austria", 5), axis=0, inplace = True)
data.drop(("Ireland", 3), axis=0, inplace = True)
data.drop(("Ireland", 4), axis=0, inplace = True)
data.loc["United Kingdom", "official"] = True
data.loc[("Albania", 7), "language"] = "Croatian"
data.loc[("Austria", 7), "language"] = "Burgenland Croatian"
data

languageRaw  \
Country        langId                                                      
Albania        0       Albanian 98.8% (official - derived from Tosk d...   
               1                                              Greek 0.5%   
               2                        other 0.6% (including Macedonian   
               3                                                  Romani   
               4                                                   Vlach   
...                                                                  ...   
Ukraine        1                       Russian (regional language) 29.6%   
               2                    other (includes small Crimean Tatar-   
               3                                      Moldovan/Romanian-   
               4        and Hungarian-speaking minorities) 2.9% (2001...   
United Kingdom 0                                                 English   

                       official    language  
Country        langId                        
Albania        0           True    Albanian  
               1          False       Greek  
               2          False  Macedonian  
               3          False      Romani  
               4          False       Vlach  
...                         ...         ...  
Ukraine        1          False     Russian  
               2          False     Crimean  
               3          False    Moldovan  
               4          False   Hungarian  
United Kingdom 0           True     English  

[148 rows x 3 columns]

In [13]:
# save processed data
data.to_csv(os.path.join(PATH_DATA, "general", "countries_languages_eur.csv"), index=True)

In [14]:
# get endangerment status
endangered = pd.read_csv("../../data/endangerment/endangered_languages_europe.csv")

In [15]:
# join with endangered_languages_europe.csv
endangered = endangered[["Name in English", "Degree of endangerment","Endangerment code", "Number of speakers"]]
endangered.head()

,Name in English,Degree of endangerment,Endangerment code,Number of speakers
0,South Italian,Vulnerable,1,7500000.0
1,Sicilian,Vulnerable,1,5000000.0
2,Low Saxon,Vulnerable,1,4800000.0
3,Belarusian,Vulnerable,1,4000000.0
4,Lombard,Definitely endangered,2,3500000.0


In [17]:
# keep index -> back to columns
data.reset_index(inplace=True)
dataEndangered = pd.merge(data, endangered, how='left', left_on=['language'], right_on=["Name in English"])
#data #.head()


In [18]:
dataEndangered

,Country,langId,languageRaw,official,language,Name in English,Degree of endangerment,Endangerment code,Number of speakers
0,Albania,0,Albanian 98.8% (official - derived from Tosk d...,True,Albanian,NaN,NaN,NaN,NaN
1,Albania,1,Greek 0.5%,False,Greek,NaN,NaN,NaN,NaN
2,Albania,2,other 0.6% (including Macedonian,False,Macedonian,NaN,NaN,NaN,NaN
3,Albania,3,Romani,False,Romani,Romani,Definitely endangered,2.0,3500000.0
4,Albania,4,Vlach,False,Vlach,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
143,Ukraine,1,Russian (regional language) 29.6%,False,Russian,NaN,NaN,NaN,NaN
144,Ukraine,2,other (includes small Crimean Tatar-,False,Crimean,NaN,NaN,NaN,NaN
145,Ukraine,3,Moldovan/Romanian-,False,Moldovan,NaN,NaN,NaN,NaN
146,Ukraine,4,and Hungarian-speaking minorities) 2.9% (2001...,False,Hungarian,NaN,NaN,NaN,NaN


In [19]:
# define class: Endangerment code NaN -> 0 (not endangered), integer -> 1 (endangered)
dataEndangered['endangered'] = [0 if pd.isna(ec) else 1 for ec in dataEndangered['Endangerment code']]
dataEndangered

,Country,langId,languageRaw,official,language,Name in English,Degree of endangerment,Endangerment code,Number of speakers,endangered
0,Albania,0,Albanian 98.8% (official - derived from Tosk d...,True,Albanian,NaN,NaN,NaN,NaN,0
1,Albania,1,Greek 0.5%,False,Greek,NaN,NaN,NaN,NaN,0
2,Albania,2,other 0.6% (including Macedonian,False,Macedonian,NaN,NaN,NaN,NaN,0
3,Albania,3,Romani,False,Romani,Romani,Definitely endangered,2.0,3500000.0,1
4,Albania,4,Vlach,False,Vlach,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
143,Ukraine,1,Russian (regional language) 29.6%,False,Russian,NaN,NaN,NaN,NaN,0
144,Ukraine,2,other (includes small Crimean Tatar-,False,Crimean,NaN,NaN,NaN,NaN,0
145,Ukraine,3,Moldovan/Romanian-,False,Moldovan,NaN,NaN,NaN,NaN,0
146,Ukraine,4,and Hungarian-speaking minorities) 2.9% (2001...,False,Hungarian,NaN,NaN,NaN,NaN,0


In [30]:
# join with European languages for country code
eur_lang = eur_lang[["CountryID", "Name_country"]]
eur_lang = eur_lang.drop_duplicates()
eur_lang.head()

,CountryID,Name_country
0,IT,Italy
22,GR,Greece
31,RU,Russian Federation
124,NL,Netherlands
136,AL,Albania


In [31]:
dataLang = pd.merge(dataEndangered, eur_lang, how='left', left_on=['Country'], right_on=["Name_country"])
dataLang.head()

,Country,langId,languageRaw,official,language,Name in English,Degree of endangerment,Endangerment code,Number of speakers,endangered,CountryID,Name_country
0,Albania,0,Albanian 98.8% (official - derived from Tosk d...,True,Albanian,NaN,NaN,NaN,NaN,0,AL,Albania
1,Albania,1,Greek 0.5%,False,Greek,NaN,NaN,NaN,NaN,0,AL,Albania
2,Albania,2,other 0.6% (including Macedonian,False,Macedonian,NaN,NaN,NaN,NaN,0,AL,Albania
3,Albania,3,Romani,False,Romani,Romani,Definitely endangered,2.0,3500000.0,1,AL,Albania
4,Albania,4,Vlach,False,Vlach,NaN,NaN,NaN,NaN,0,AL,Albania


In [32]:
dataLang.drop(columns =["Name_country"], inplace=True)

In [33]:
dataLang

,Country,langId,languageRaw,official,language,Name in English,Degree of endangerment,Endangerment code,Number of speakers,endangered,CountryID
0,Albania,0,Albanian 98.8% (official - derived from Tosk d...,True,Albanian,NaN,NaN,NaN,NaN,0,AL
1,Albania,1,Greek 0.5%,False,Greek,NaN,NaN,NaN,NaN,0,AL
2,Albania,2,other 0.6% (including Macedonian,False,Macedonian,NaN,NaN,NaN,NaN,0,AL
3,Albania,3,Romani,False,Romani,Romani,Definitely endangered,2.0,3500000.0,1,AL
4,Albania,4,Vlach,False,Vlach,NaN,NaN,NaN,NaN,0,AL
...,...,...,...,...,...,...,...,...,...,...,...
143,Ukraine,1,Russian (regional language) 29.6%,False,Russian,NaN,NaN,NaN,NaN,0,UA
144,Ukraine,2,other (includes small Crimean Tatar-,False,Crimean,NaN,NaN,NaN,NaN,0,UA
145,Ukraine,3,Moldovan/Romanian-,False,Moldovan,NaN,NaN,NaN,NaN,0,UA
146,Ukraine,4,and Hungarian-speaking minorities) 2.9% (2001...,False,Hungarian,NaN,NaN,NaN,NaN,0,UA


In [ ]:
# NEXT: join with population, immigrants, emigrants where year = first and last (or take mean/median?!)

In [34]:
# save joined tables
dataLang.to_csv(os.path.join(PATH_DATA, "general", "flat_table.csv"), index=True)